In [1]:
#run this
import os
import random
import urllib.request
import zipfile
from pathlib import Path
from tqdm import tqdm
import pandas as pd
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import torch
import numpy as np
from tqdm import tqdm

E:\clip_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
#run this
import zipfile
from pathlib import Path

# Set path to your uploaded ZIP file and output folder
zip_path = Path("flickr_data/Flickr8k_Dataset.zip")  # change if it's elsewhere
extract_to = Path("flickr_data")  # you can change this too

# Function to unzip
def unzip(zip_path, extract_to):
    print(f"📦 Unzipping {zip_path.name}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Done extracting!")

# Call it
unzip(zip_path, extract_to)


📦 Unzipping Flickr8k_Dataset.zip...
✅ Done extracting!


In [2]:
#run
import pandas as pd

# Set your paths
CAPTION_FILE = "flickr_data/Flickr8k.token.txt"
IMG_DIR = "flickr_data/Flicker8k_Dataset"  # spelling check here!

# Load captions
with open(CAPTION_FILE, 'r') as f:
    lines = f.readlines()

# Parse captions into a DataFrame
data = []
for line in lines:
    img_caption = line.strip().split('\t')
    img_filename = img_caption[0].split('#')[0]  # Getting the filename (e.g., image.jpg)
    caption = img_caption[1]
    data.append((img_filename, caption))

df = pd.DataFrame(data, columns=["image", "caption"])

# Group captions by image
grouped = df.groupby("image")["caption"].apply(list).reset_index()

# Now, we have all 1000 image-caption pairs
print("✅ All image-caption pairs:", grouped.head())


✅ All image-caption pairs:                        image  \
0  1000268201_693b08cb0e.jpg   
1  1001773457_577c3a7d70.jpg   
2  1002674143_1b742ab4b8.jpg   
3  1003163366_44323f5815.jpg   
4  1007129816_e794419615.jpg   

                                             caption  
0  [A child in a pink dress is climbing up a set ...  
1  [A black dog and a spotted dog are fighting, A...  
2  [A little girl covered in paint sits in front ...  
3  [A man lays on a bench while his dog sits by h...  
4  [A man in an orange hat starring at something ...  


In [2]:
pip install transformers torchvision faiss-cpu gradio


^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os._exit(00)  # This will exit the kernel


In [3]:
#run
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import torch
import numpy as np
from tqdm import tqdm

from transformers import CLIPProcessor, CLIPModel

# Load CLIP model from OpenAI
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
model.eval()  # set to evaluation mode



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [6]:
pip install torch torchvision torchaudio


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
os._exit(00)  # This will exit the kernel


In [4]:
img_dir = "E:/intel/flickr_data/Flicker8k_Dataset"
token_path = "E:/intel/flickr_data/Flickr8k.token.txt"
import os

print("Image dir exists:", os.path.exists(img_dir))  # should be True
print("Token file exists:", os.path.exists(token_path))  # should be True


Image dir exists: True
Token file exists: True


In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
import os
from tqdm import tqdm
import glob

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Set paths
img_dir = "E:/intel/flickr_data/Flicker8k_Dataset"
token_path = "E:/intel/flickr_data/Flickr8k.token.txt"
save_dir = "E:/intel/flickr_embeddings"
os.makedirs(save_dir, exist_ok=True)

# Load token file
with open(token_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Parse into (img_path, caption) pairs
pairs = []
for line in lines:
    img_id, caption = line.strip().split('\t')
    img_file = img_id.split('#')[0]
    img_path = os.path.join(img_dir, img_file)
    if os.path.exists(img_path):
        pairs.append((img_path, caption))

print(f"✅ Total pairs available: {len(pairs)}")

# Batch loop — resume from batch 9
batch_size = 500
start_batch = 9

for i in range(start_batch, (len(pairs) // batch_size) + 1):
    start = i * batch_size
    end = start + batch_size
    batch = pairs[start:end]

    image_embeddings = []
    text_embeddings = []

    for img_path, caption in tqdm(batch, desc=f"Batch {i+1}"):
        try:
            image = Image.open(img_path).convert("RGB")
            inputs = processor(text=caption, images=image, return_tensors="pt", padding=True).to(device)
            outputs = model(**inputs)

            image_embed = outputs.image_embeds.detach().cpu()
            text_embed = outputs.text_embeds.detach().cpu()

            image_embeddings.append(image_embed)
            text_embeddings.append(text_embed)
        except Exception as e:
            print(f"⚠️ Skipping {img_path}: {e}")

    if image_embeddings:
        torch.save(torch.cat(image_embeddings), os.path.join(save_dir, f"image_embeddings_batch_{i+1}.pt"))
    if text_embeddings:
        torch.save(torch.cat(text_embeddings), os.path.join(save_dir, f"text_embeddings_batch_{i+1}.pt"))

    print(f"✅ Saved batch {i+1} embeddings")


✅ Total pairs available: 40455


Batch 10: 100%|████████████████████████████████████████████████████| 500/500 [04:32<00:00,  1.84it/s]


✅ Saved batch 10 embeddings


Batch 11: 100%|████████████████████████████████████████████████████| 500/500 [04:47<00:00,  1.74it/s]


✅ Saved batch 11 embeddings


...

Batch 62: 100%|████████████████████████████████████████████████████| 500/500 [04:06<00:00,  2.03it/s]


✅ Saved batch 62 embeddings


Batch 63: 100%|████████████████████████████████████████████████████| 500/500 [03:51<00:00,  2.16it/s]


✅ Saved batch 63 embeddings


Batch 64:  39%|████████████████████▍                               | 196/500 [01:26<02:39,  1.90it/s]

In [23]:
import os
import glob
from PIL import Image
from torchvision import transforms

# Set paths
img_dir = "E:/intel/flickr_data/Flicker8k_Dataset"
token_path = "E:/intel/flickr_data/Flickr8k.token.txt"

# Load all image paths
img_paths = sorted(glob.glob(os.path.join(img_dir, "*.jpg")))

# Load captions into a dict: {filename: [list of captions]}
caption_dict = {}
with open(token_path, "r") as f:
    for line in f:
        fname, caption = line.strip().split("\t")
        fname = fname.split("#")[0]  # remove the #0, #1, etc.
        if fname not in caption_dict:
            caption_dict[fname] = []
        caption_dict[fname].append(caption)

# Build all_images and all_texts using one caption per image
all_images = []
all_texts = []

for img_path in img_paths:
    fname = os.path.basename(img_path)
    if fname in caption_dict:
        all_images.append(Image.open(img_path).convert("RGB"))
        all_texts.append(caption_dict[fname][0])  # Use first caption


In [24]:
import torch
import glob

img_paths = sorted(glob.glob("E:/intel/image_embeddings_batch_*.pt"))
txt_paths = sorted(glob.glob("E:/intel/text_embeddings_batch_*.pt"))

print(f"Found {len(img_paths)} image embedding files.")
print(f"Found {len(txt_paths)} text embedding files.")

image_embeddings = []
text_embeddings = []

# Load image embeddings
for p in img_paths:
    emb = torch.load(p)
    if emb.numel() > 0:
        image_embeddings.append(emb)

# Load text embeddings
for p in txt_paths:
    emb = torch.load(p)
    if emb.numel() > 0:
        text_embeddings.append(emb)

import torch

image_embeddings = torch.cat(image_embeddings)
text_embeddings = torch.cat(text_embeddings)

print("✅ Loaded image embeddings shape:", image_embeddings.shape)
print("✅ Loaded text embeddings shape:", text_embeddings.shape)



Found 8 image embedding files.
Found 8 text embedding files.
✅ Loaded image embeddings shape: torch.Size([4000, 512])
✅ Loaded text embeddings shape: torch.Size([4000, 512])


In [6]:
import faiss
import os

# Convert to float32 numpy (FAISS only supports float32)
img_emb_np = image_embeddings.detach().cpu().numpy().astype('float32')

# Create the FAISS index
d = img_emb_np.shape[1]  # embedding dimension
index = faiss.IndexFlatL2(d)  # L2 = Euclidean distance
index.add(img_emb_np)  # add image embeddings

# Save the index
faiss.write_index(index, "E:/intel/flickr_faiss_index.idx")

print("🎯 FAISS index created and saved!")


🎯 FAISS index created and saved!


In [9]:
import gc
import torch

# Clear unused variables
del image_embeddings, text_embeddings

# Garbage collection
gc.collect()

# Empty GPU cache if using CUDA (not likely on CPU but just in case)
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("✅ RAM cleaned up!")


NameError: name 'image_embeddings' is not defined

E:\clip_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Image embeddings collected: 0


❌ Deleted corrupted zip file.


In [20]:
pip install faiss-cpu


  Using cached faiss_cpu-1.10.0-cp310-cp310-win_amd64.whl (13.7 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
pip install gradio

  Using cached gradio-5.23.3-py3-none-any.whl (46.5 MB)
  Using cached orjson-3.10.16-cp310-cp310-win_amd64.whl (133 kB)
  Using cached typer-0.15.2-py3-none-any.whl (45 kB)
     ---------------------------------------- 95.2/95.2 kB 2.7 MB/s eta 0:00:00
  Using cached tomlkit-0.13.2-py3-none-any.whl (37 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached ruff-0.11.3-py3-none-win_amd64.whl (11.4 MB)
     -------------------------------------- 322.2/322.2 kB 6.6 MB/s eta 0:00:00
  Using cached groovy-0.1.2-py3-none-any.whl (14 kB)
     ------------------------------------- 443.3/443.3 kB 14.0 MB/s eta 0:00:00
  Using cached safehttpx-0.1.6-py3-none-any.whl (8.7 kB)
     ---------------------------------------- 62.3/62.3 kB 3.3 MB/s eta 0:00:00
  Using cached starlette-0.46.1-py3-none-any.whl (71 kB)
     -------------------------------------- 176.8/176.8 kB 5.4 MB/s eta 0:00:00
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
     ---------


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
